In [1]:
/* 22-14 */

USE AdventureWorks2014;
GO
SELECT cp.objtype, AdHocText
FROM sys.dm_exec_cached_plans cp
CROSS APPLY (SELECT text AS [processing-instruction(definition)]
            FROM sys.dm_exec_sql_text(cp.plan_handle) st
            WHERE st.text LIKE 'SELECT BusinessEntityID%'
            FOR XML PATH(''), TYPE
      ) AS st(AdHocText)
WHERE st.AdHocText IS NOT NULL;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.013

objtype,AdHocText


In [2]:
/* Now let's create a template */
DECLARE @sql  nvarchar(max) DECLARE @parms nvarchar(max)
EXEC sp_get_query_template
N'SELECT BusinessEntityID FROM HumanResources.Employee WHERE NationalIDNumber = 295847284',
@sql OUTPUT,
@parms OUTPUT;

EXEC sp_create_plan_guide N'PG_Employee_Contact_Ouery', @sql,
N'TEMPLATE', NULL, @parms, N'OPTION(PARAMETERIZATION FORCED)';

Commands completed successfully.

Total execution time: 00:00:00.025

In [3]:
/* Now try those queries again */
USE AdventureWorks2014;
GO
SELECT BusinessEntityID
        FROM HumanResources.Employee
        WHERE NationalIDNumber = 509647174;
GO
SELECT BusinessEntityID
        FROM HumanResources.Employee
        WHERE NationalIDNumber = 245797967;
GO
SELECT BusinessEntityID
        FROM HumanResources.Employee
        WHERE NationalIDNumber = 295847284;
GO

Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.062

BusinessEntityID
3


BusinessEntityID
1


In [4]:
/* check for a prepared plan now */
USE AdventureWorks2014;
GO
SELECT usecounts,objtype,PreparedText 
FROM sys.dm_exec_cached_plans cp 
CROSS APPLY (SELECT text AS [processing-instruction(definition)]
            FROM sys.dm_exec_sql_text(cp.plan_handle) st
            WHERE st.text LIKE '%SELECT BusinessEntityID%'
            FOR XML PATH(''), TYPE
      ) AS st(PreparedText) 
WHERE st.PreparedText IS NOT NULL
AND objtype = 'Prepared';

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.702

usecounts,objtype,PreparedText
3,Prepared,<?definition (@0 int)select BusinessEntityID from HumanResources . Employee where NationalIDNumber = @0?>
